### loading dataset

In [129]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)
data = pd.read_csv(r'../2. data_cleaning\4.1 some cols FEed.csv')
data.head(3)

,property_name,sector,society,area_in_sqft,price_in_crores,price_per_sqft,areaWithType,bedRoom,bathroom,balcony,additionalRoom,address,floorNum,facing,agePossession,nearbyLocations,description,furnishDetails,features,rating,property_type,noOfFloor,area
0,2 BHK Flat in Sector 70 Gurgaon,sector 70,shree vardhman victoria,1300.0,1.10,8461.0,Carpet area: 1300 (120.77 sq.m.),2.0,2.0,2,not available,"Sector 70 Gurgaon, Gurgaon, Haryana",8.0,north-east,1 to 5 Year Old,"['Airia Mall', 'Golf Course Extn Road', 'Sohna Road', 'Southern Peripheral Rd, Gurugram', 'Kunskapsskolan International', 'Ektaa Hospitals Main Sohna Rd', 'Sanjeevani Hospital - Child Specialist', 'Federal Bank Sector 71', 'Central Bank Of India Sohna Rd', 'Axis Bank, Sohna Rd', 'Spaze Palazo, Golf Course Ext Rd', 'VATIKA BUSINESS PARK Sohna Rd', 'The Medicity, Spaze iTech Park', 'Tulip Violet Society, Sector 69', 'IndianOil, Hasanpur']","Square yards rera reg.\r\nHrera 660/2017/307\r\nProject rera reg.\r\n70 of 2017\r\nShree vardhman victoria - Project info\r\nShree vardhman victoria is an exclusive project located in sector 70, southern peripheral road and well connected by major road(s) like sohna road, southern peripheral road. The total area in which shree vardhman victoria has been built is 11 acre. This project has been developed by shree vardhman who are one of the reputed developers in the gurgaon. The project current status is ready to move. It has 592 units. The apartment units are available in various configurations like 2 bhk flats at a price of rs 82.44 l with property size of 1300 sqft, 3 bhk flats at a price of rs 1.24 cr with property size of 1950 sqft, 4 bhk flats at a price of rs 1.62 cr with property size of 2550 sqft.\r\nShree vardhman victoria - Price list\r\nUnit type area new home price*\r\n2 bhk 1300 sq. Ft.\r\nApartment \r\n1300 sq. Ft.\r\n(Saleable)\r\n 82.44 lac\r\n3 bhk 1950 sq. Ft.\r\nA...","['3 Wardrobe', '1 Modular Kitchen', '1 Chimney', 'No AC', 'No Bed', 'No Curtains', 'No Dining Table', 'No Exhaust Fan', 'No Fan', 'No Geyser', 'No Light', 'No Microwave', 'No Fridge', 'No Sofa', 'No Stove', 'No TV', 'No Washing Machine', 'No Water Purifier']","['Security / Fire Alarm', 'Power Back-up', 'Feng Shui / Vaastu Compliant', 'Intercom Facility', 'Lift(s)', 'High Ceiling Height', 'Maintenance Staff', 'False Ceiling Lighting', 'Water Storage', 'Separate entry for servant room', 'No open drainage around', 'Piped-gas', 'Swimming Pool', 'Park', 'Internet/wi-fi connectivity', 'Shopping Centre', 'Fitness Centre / GYM', 'Waste Disposal', 'Rain Water Harvesting', 'Club house / Community Center', 'Water softening plant']","['Green Area5 out of 5', 'Construction5 out of 5', 'Management4.5 out of 5', 'Amenities4.5 out of 5', 'Connectivity3.5 out of 5']",flat,1.0,NaN
1,3 Bedroom House for sale in Sector 40 Gurgaon,sector 40,NaN,900.0,2.15,23889.0,Plot area 100(83.61 sq.m.),3.0,3.0,3,"Pooja Room,Study Room,Store Room","Sector 40 Gurgaon, Gurgaon, Haryana",0.0,NaN,5 to 10 year old,"['Huda city centre metro station', 'Axis bank ATM', 'State bank of india ATM', 'Icici bank ATM', 'Icici ATM', 'Citi bank ATM', 'Axis bank ATM', 'Hdfc ATM', 'Standard chartered ATM', 'Hdfc bank ATM', 'Axis bank ATM', 'Hdfc bank ATM', 'State bank of india ATM', 'Dispencery', 'Shivam Hospital Gurgaon', 'Fortis Memorial Research Institute Fortis Vivekanand Hospital', 'Dr. Naval Mendiratta', 'Centre For Sight Gurgaon Sector 29', 'Sukhmani Hospital Pvt. Ltd', 'Ahmed Hospital Multi Speciality', ""DR AKRAM JAWED'S THE UPPER LIMB CLINIC"", 'Medanta', 'Gardian Pharmacy', 'City Medical', 'Gardian Pharmacy', 'Pernod Ricard Charitable Dispensary', 'IBP Petrol Pump', 'Bharat petroleum', 'Hdfc bank', 'Raj Restaurant', 'Fast food', 'Fast Food', 'Om Sweets', 'Bar and restaurant', 'Cafe Coffee Day', '32nd Milestone', 'Darbar', 'PWO house', 'Dhabba', 'St. Angels Jr', 'School of Inspired Leadership SOIL', 'CR Model Public School', 'Manav Rachna School', 'St. Angels Sr', 'Amity Global School', 'Salvan

### FEing `areaWithType` col

In [130]:
data['areaWithType'].head(3)

0    Carpet area: 1300 (120.77 sq.m.)
1          Plot area 100(83.61 sq.m.)
2         Plot area 520(434.79 sq.m.)
Name: areaWithType, dtype: object

In [131]:
data['areaWithType'] = data['areaWithType'].str.strip().str.lower()

In [132]:
x = data['areaWithType'].value_counts(dropna=False)
x[x>10]

areaWithType
NaN                                                                              108
plot area 360(301.01 sq.m.)                                                       39
plot area 300(250.84 sq.m.)                                                       27
plot area 200(167.23 sq.m.)                                                       21
plot area 502(419.74 sq.m.)                                                       19
plot area 270(225.75 sq.m.)                                                       19
super built up area 1578(146.6 sq.m.)                                             17
super built up area 1350(125.42 sq.m.)                                            17
super built up area 1950(181.16 sq.m.)carpet area: 1161 sq.ft. (107.86 sq.m.)     17
plot area 900(83.61 sq.m.)                                                        17
plot area 500(418.06 sq.m.)                                                       15
super built up area 1650(153.29 sq.m.)carpet area: 1

#### `area_in_sqft` col contains one of the 3 area_types in `areaWithType` col.

In [133]:
data[['area_in_sqft', 'areaWithType']].sample(3)

,area_in_sqft,areaWithType
682,1790.0,super built up area 1790(166.3 sq.m.)carpet area: 1350 sq.ft. (125.42 sq.m.)
2173,3240.0,plot area 360(301.01 sq.m.)
3789,700.0,super built up area 700(65.03 sq.m.)carpet area: 450 sq.ft. (41.81 sq.m.)


#### creating `superBuiltupArea` col

In [134]:
# areaWithType containing "super" keyword
data[data['areaWithType'].str.lower().str.strip().fillna('').str.contains('super')]['areaWithType']

3                                                                                       super built up area 2010(186.74 sq.m.)
11                                                                                      super built up area 2217(205.97 sq.m.)
15                                                                                        super built up area 700(65.03 sq.m.)
16      super built up area 1424(132.29 sq.m.)built up area: 1200 sq.ft. (111.48 sq.m.)carpet area: 1139 sq.ft. (105.82 sq.m.)
17                                                                                      super built up area 1960(182.09 sq.m.)
                                                                 ...                                                          
4068                                             super built up area 1990(184.88 sq.m.)carpet area: 1700 sq.ft. (157.94 sq.m.)
4071                                                                                    super built up area 195

In [135]:
data['SuperBuiltupArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'super built up area (\d+\.?\d+)')
data['SuperBuiltupArea']

0        NaN
1        NaN
2        NaN
3       2010
4        NaN
        ... 
4077     NaN
4078    1780
4079     NaN
4080     NaN
4081     NaN
Name: SuperBuiltupArea, Length: 4082, dtype: object

In [136]:
data['SuperBuiltupArea'].value_counts(dropna=False)[data['SuperBuiltupArea'].value_counts(dropna=False) >20]

SuperBuiltupArea
NaN     2148
1650      38
1950      38
2000      26
1578      25
2150      23
1640      22
Name: count, dtype: int64

In [137]:
# observation
# 2148 places regex couldn't extract superBuiltupArea value.

In [138]:
data[data['SuperBuiltupArea'].isna()][['SuperBuiltupArea', 'areaWithType']].sample(5)

,SuperBuiltupArea,areaWithType
2019,NaN,built up area: 1734 (161.09 sq.m.)
3868,NaN,carpet area: 2290 (212.75 sq.m.)
2964,NaN,built up area: 800 (74.32 sq.m.)
3076,NaN,carpet area: 1690 (157.01 sq.m.)
3276,NaN,plot area 521


#### creating `BuiltupArea` col

In [139]:
data['areaWithType'].str.lower().str.strip().sample(3)

2978    built up area: 1081 (100.43 sq.m.)
1137             plot area 14(354.1 sq.m.)
3920      carpet area: 1900 (176.52 sq.m.)
Name: areaWithType, dtype: object

In [140]:
data['BuiltupArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'built up area\s*:\s*(\d+\.?\d*)')
data['BuiltupArea']

0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
       ... 
4077    NaN
4078    NaN
4079    NaN
4080    NaN
4081    NaN
Name: BuiltupArea, Length: 4082, dtype: object

In [141]:
data['BuiltupArea'].value_counts(dropna=False)[data['BuiltupArea'].value_counts(dropna=False) >20]

BuiltupArea
NaN     2848
1900      34
1600      26
2000      25
1300      24
1700      22
Name: count, dtype: int64

In [142]:
# observation
# 2848 places regex couldn't extract BuiltupArea value.

In [143]:
data[data['BuiltupArea'].isna()][['BuiltupArea', 'areaWithType']].sample(5)

,BuiltupArea,areaWithType
3394,NaN,plot area 200(167.23 sq.m.)
1978,NaN,NaN
3118,NaN,super built up area 1385(128.67 sq.m.)
1530,NaN,NaN
876,NaN,carpet area: 675 (62.71 sq.m.)


#### creating `CarpetArea` col

In [144]:
data['areaWithType'].str.lower().str.strip().sample(3)

3113                                                                                              plot area 2250(209.03 sq.m.)
2145                                                                                    super built up area 2048(190.27 sq.m.)
547     super built up area 1805(167.69 sq.m.)built up area: 1350 sq.ft. (125.42 sq.m.)carpet area: 1100 sq.ft. (102.19 sq.m.)
Name: areaWithType, dtype: object

In [145]:
data['CarpetArea'] = data['areaWithType'].str.lower().str.strip().str.extract(r'carpet area\s*:\s*(\d+\.?\d*)')
data['CarpetArea']

0       1300
1        NaN
2        NaN
3        NaN
4       3500
        ... 
4077     NaN
4078     NaN
4079    2295
4080    1120
4081    1300
Name: CarpetArea, Length: 4082, dtype: object

In [146]:
data['CarpetArea'].value_counts(dropna=False)[data['CarpetArea'].value_counts(dropna=False) >20]

CarpetArea
NaN     2079
1400      42
1800      37
1600      36
1200      33
1500      30
1350      29
1650      28
900       25
2000      23
1300      23
1000      23
1450      23
2200      21
Name: count, dtype: int64

In [147]:
# observation
# 2079 places regex couldn't extract CarpetArea value.

In [148]:
data[data['CarpetArea'].isna()][['CarpetArea', 'areaWithType']].sample(5)

,CarpetArea,areaWithType
2153,NaN,super built up area 1262(117.24 sq.m.)
1004,NaN,plot area 360(301.01 sq.m.)
3179,NaN,super built up area 1479(137.4 sq.m.)
202,NaN,plot area 100(83.61 sq.m.)
2950,NaN,plot area 205(171.41 sq.m.)


#### converting all 3 area_cols from sqm to sqft

In [149]:
data[['areaWithType', 'SuperBuiltupArea',	'BuiltupArea',	'CarpetArea']].sample(5)

,areaWithType,SuperBuiltupArea,BuiltupArea,CarpetArea
3573,super built up area 1963(182.37 sq.m.)carpet area: 1700 sq.ft. (157.94 sq.m.),1963,NaN,1700
332,super built up area 1630(151.43 sq.m.),1630,NaN,NaN
4055,carpet area: 1444 (134.15 sq.m.),NaN,NaN,1444
3254,carpet area: 2320 (215.54 sq.m.),NaN,NaN,2320
3959,super built up area 1852(172.06 sq.m.)carpet area: 1128.09 sq.ft. (104.8 sq.m.),1852,NaN,1128.09


In [150]:
import re
def convert_to_sqft(area_text, area_value):
    if not area_value:
        return None
    match = re.search(r"{} \((\d+\.?\d*) sq.m.\)*".format(area_value), str(area_text))
    if match:
        sqm_value = float(match.group(1))
        return sqm_value*10.76391
    return area_value

In [151]:
data['CarpetArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['CarpetArea']), axis=1)
data['BuiltupArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['BuiltupArea']), axis=1)
data['SuperBuiltupArea'] = data.apply(lambda x: convert_to_sqft(x['areaWithType'], x['SuperBuiltupArea']), axis=1)

In [152]:
data[['SuperBuiltupArea', 'BuiltupArea',	'CarpetArea']].head(3)

,SuperBuiltupArea,BuiltupArea,CarpetArea
0,NaN,NaN,1299.957411
1,NaN,NaN,NaN
2,NaN,NaN,NaN


#### 1) checking where none of the 3 area values are present

In [153]:
data[(data['CarpetArea'].isnull() & data['BuiltupArea'].isnull() & data['SuperBuiltupArea'].isnull())][['CarpetArea', 'BuiltupArea', 'SuperBuiltupArea']].shape[0]

735

In [154]:
data.shape[0]

4082

In [155]:
data[(data['CarpetArea'].isnull() & data['BuiltupArea'].isnull() & data['SuperBuiltupArea'].isnull())][['CarpetArea', 'BuiltupArea', 'SuperBuiltupArea']].sample(5)

,CarpetArea,BuiltupArea,SuperBuiltupArea
896,NaN,NaN,NaN
2389,NaN,NaN,NaN
705,NaN,NaN,NaN
718,NaN,NaN,NaN
3985,NaN,NaN,NaN


In [156]:
# there are 737 rows with no are at all, it is becoz some rows also have plot_area which we are yet to extract

##### 1.1) extracting `plot_area` values from areaWithType col

In [157]:
data['areaWithType']

0             carpet area: 1300 (120.77 sq.m.)
1                   plot area 100(83.61 sq.m.)
2                  plot area 520(434.79 sq.m.)
3       super built up area 2010(186.74 sq.m.)
4             carpet area: 3500 (325.16 sq.m.)
                         ...                  
4077               plot area 300(250.84 sq.m.)
4078    super built up area 1780(165.37 sq.m.)
4079          carpet area: 2295 (213.21 sq.m.)
4080          carpet area: 1120 (104.05 sq.m.)
4081          carpet area: 1300 (120.77 sq.m.)
Name: areaWithType, Length: 4082, dtype: object

In [158]:
data[data['areaWithType'].fillna('').str.contains('plot')]['areaWithType']

1                                                                                          plot area 100(83.61 sq.m.)
2                                                                                         plot area 520(434.79 sq.m.)
8       plot area 260(217.39 sq.m.)built up area: 250 sq.yards (209.03 sq.m.)carpet area: 245 sq.yards (204.85 sq.m.)
9                                                                                          plot area 263(219.9 sq.m.)
10                                                                                          plot area 75(62.71 sq.m.)
                                                            ...                                                      
4048                                                                                      plot area 168(140.47 sq.m.)
4050                                                                                     plot area 1000(836.13 sq.m.)
4057                                                    

In [159]:
data['plotArea'] = data['areaWithType'].str.extract(r'plot area \d+\.*\d*\((\d+\.*\d*)')
data['plotArea']

0          NaN
1        83.61
2       434.79
3          NaN
4          NaN
         ...  
4077    250.84
4078       NaN
4079       NaN
4080       NaN
4081       NaN
Name: plotArea, Length: 4082, dtype: object

##### checking with a simple_regex if the my complex_regex was able to extract ALL the records correctly or not:

In [160]:
data[(data['plotArea'].isnull()) & (data['areaWithType'].fillna('').str.contains('plot'))][['areaWithType', 'plotArea']].shape[0]

7

In [161]:
data[(data['plotArea'].isnull()) & (data['areaWithType'].fillna('').str.contains('plot'))][['areaWithType', 'plotArea']]

,areaWithType,plotArea
896,plot area 754,NaN
1969,plot area 160,NaN
2695,plot area 50,NaN
2947,plot area 60built up area: 60 sq.m.carpet area: 59 sq.m.,NaN
3267,plot area 220built up area: 170 sq.m.carpet area: 160 sq.m.,NaN
3276,plot area 521,NaN
3863,plot area 420,NaN


In [162]:
# observation:
    # here not every plot are record contained the sqm area, so i had to go with the number before it and to understand what is the unit of this number, i did chatgpt like this:
        # Sarvesh Kesharwani:
            # what is the number before brackets "carpet area: 2295 (213.21 sq.m.)"
        # chatgpt:
            # The number 2295 represents the carpet area in square feet (sq ft).
            # The information within the brackets, "(213.21 sq.m.)", provides the equivalent area in square meters (sq m).

    # and found that it is sqft area.

##### extracting plot_area again but sqft unit this time instead of sqm

In [163]:
data['plotArea'] = data['areaWithType'].str.extract(r'plot area (\d+\.?\d*)')
data['plotArea']

0       NaN
1       100
2       520
3       NaN
4       NaN
       ... 
4077    300
4078    NaN
4079    NaN
4080    NaN
4081    NaN
Name: plotArea, Length: 4082, dtype: object

##### again checking with a simple_regex if the my complex_regex was able to extract ALL the records correctly or not:

In [164]:
data[(data['plotArea'].isnull()) & (data['areaWithType'].fillna('').str.contains('plot'))][['areaWithType', 'plotArea']].shape[0]

0

In [165]:
data[(data['plotArea'].isnull()) & (data['areaWithType'].fillna('').str.contains('plot'))][['areaWithType', 'plotArea']]

,areaWithType,plotArea


In [166]:
# okay so this time it was successfull.

##### now let's merge plot area with builtupArea, since according to GPT, builtupArea of a flat is most simmilar to plotArea in case of independentHouses.

In [167]:
data[['BuiltupArea', 'plotArea']]

,BuiltupArea,plotArea
0,NaN,NaN
1,NaN,100
2,NaN,520
3,NaN,NaN
4,NaN,NaN
...,...,...
4077,NaN,300
4078,NaN,NaN
4079,NaN,NaN
4080,NaN,NaN


##### checking if for any record i have both the plot and builtup areas causing conflict while merging.

In [168]:
data[(~data['BuiltupArea'].isna()) & (~data['plotArea'].isna())].shape[0]

141

In [169]:
pd.set_option('display.max_colwidth', 50)
data[(~data['BuiltupArea'].isna()) & (~data['plotArea'].isna())][['BuiltupArea', 'plotArea']]

,BuiltupArea,plotArea
8,250,260
30,61,100
54,1415,173
103,3750,135
146,9000,4140
...,...,...
3896,120,120
3956,800,900
3980,7500,1623
3984,345,181.33


In [170]:
data['areaWithType']

0             carpet area: 1300 (120.77 sq.m.)
1                   plot area 100(83.61 sq.m.)
2                  plot area 520(434.79 sq.m.)
3       super built up area 2010(186.74 sq.m.)
4             carpet area: 3500 (325.16 sq.m.)
                         ...                  
4077               plot area 300(250.84 sq.m.)
4078    super built up area 1780(165.37 sq.m.)
4079          carpet area: 2295 (213.21 sq.m.)
4080          carpet area: 1120 (104.05 sq.m.)
4081          carpet area: 1300 (120.77 sq.m.)
Name: areaWithType, Length: 4082, dtype: object